# *Machine Learning* em Reanimação Neonatal
Projeto 1 da Disciplina de Aprendizado de Máquina - PPgEEC/UFRN

## Etapa 3 - SEGREGATION

Este notebook servirá para dividir o *dataset* em conjunto de **treinamento** e de **teste**.

## 1.  Configuração do Ambiente

In [9]:
import logging
import os
import tempfile
import pandas as pd
import wandb
from sklearn.model_selection import train_test_split

In [10]:
!wandb login c4d070b0b80b59891761b50eb9912749a4af585e --relogin

wandb: Appending key for api.wandb.ai to your netrc file: /Users/ericcalasans/.netrc


## 2.  Segregação dos Dados

Criação das variáveis de apoio

In [11]:
# Tamanho do conjunto de teste(fração do dataset que é utilizada como dados de teste)
test_size = 0.3

# Coeficiente de aleatoriedade
seed = 1618

# Variável alvo(target)
stratify = 'reanimacao'

# Artefato de entrada
input_artifact = 'mlreanimacao/clean_data.csv:latest'

# Tipo do artefato
artifact_type = 'segregated_data'

Configurações do objeto de log - uso do módulo **logging**

In [12]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# Criação de um objeto para produzir os logs
logger = logging.getLogger()

Captura do artefato

In [13]:
run = wandb.init(project='mlreanimacao', job_type='split_data')

# Registra um log desta ação
logger.info("Baixando artefato...")
artifact = run.use_artifact(input_artifact)
artifact_file = artifact.file()
df_to_split = pd.read_csv(artifact_file)

25-05-2022 21:52:20 Baixando artefato...


Divisão do *dataset*

In [14]:
conjunto = {}

# Conjunto de treino(train) e de teste(test)
conjunto["train"], conjunto["test"] = train_test_split(
    df_to_split,
    test_size=test_size,
    random_state=seed,
    stratify=df_to_split['reanimacao']
)

## 3.  Criação de Artefatos

Serão criados artefatos separados para os conjuntos de treino e teste.

In [15]:
with tempfile.TemporaryDirectory() as temp_dir:
    for conj, df in conjunto.items():
        artifact_name = f"{conj}.csv"
        temp_path = os.path.join(temp_dir, artifact_name)
        logger.info(f"Salvando o {conj} dataset para o {artifact_name}")
        df.to_csv(temp_path, index=False)
        artifact = wandb.Artifact(
            name=artifact_name,
            type=artifact_type,
            description=f"{conj} parte do dataset {input_artifact} "
        )
        artifact.add_file(temp_path)
        logger.info("Log do artefato")
        run.log_artifact(artifact)
        artifact.wait()  # Espera o artefato ser logado

25-05-2022 21:52:29 Salvando o train dataset para o train.csv
25-05-2022 21:52:29 Log do artefato
25-05-2022 21:52:35 Salvando o test dataset para o test.csv
25-05-2022 21:52:35 Log do artefato


In [16]:
# Termina a execução do wandb
run.finish()